<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG_v2/LLM_google_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

In [3]:
# 1. Setup SQLite Database
def setup_db(db_name="rag_db.sqlite"):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS documents (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            content TEXT,
            embedding BLOB
        )
    """)
    conn.commit()
    return conn

In [4]:
# 2. Embed Documents and Store in DB
def embed_and_store(conn, documents, embedding_model):
    cursor = conn.cursor()
    for doc in documents:
        embedding = embedding_model.encode(doc, convert_to_tensor=True).numpy().tobytes()
        cursor.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (doc, embedding))
    conn.commit()


In [5]:
# 3. Retrieve Relevant Documents
def retrieve_documents(conn, query, embedding_model, top_k=3):
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cursor = conn.cursor()
    cursor.execute("SELECT id, content, embedding FROM documents")
    rows = cursor.fetchall()
    similarities = []
    for row in rows:
        doc_embedding = embedding_model.encode(row[1], convert_to_tensor=True) #using content to create embedding.
        similarity = util.pytorch_cos_sim(query_embedding, doc_embedding).item()
        similarities.append((similarity, row[1])) #store similarity and content.
    similarities.sort(key=lambda x: x[0], reverse=True)
    return [doc for _, doc in similarities[:top_k]]

In [6]:
# 4. Generate Answer with LLM
def generate_answer(query, retrieved_documents, llm_model):
    context = "\n".join(retrieved_documents)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\n\nAnswer:"
    answer = llm_model(prompt)[0]['generated_text']
    return answer

In [7]:
# Main Execution
def rag_pipeline(query, documents, embedding_model_name="all-mpnet-base-v2", llm_model_name="google/flan-t5-xxl"):
    conn = setup_db()
    embedding_model = SentenceTransformer(embedding_model_name)
    embed_and_store(conn, documents, embedding_model)
    retrieved_docs = retrieve_documents(conn, query, embedding_model)
    llm_model = pipeline("text2text-generation", model=llm_model_name)
    answer = generate_answer(query, retrieved_docs, llm_model)
    conn.close()
    return answer

In [8]:
# Example Usage
documents = [
    "The capital of France is Paris.",
    "The Eiffel Tower is in Paris.",
    "London is the capital of England.",
    "Dogs are loyal animals.",
    "Cats are independent animals."
]

In [9]:
query = "Where is the Eiffel Tower?"
answer = rag_pipeline(query, documents)
print(answer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


France


In [10]:
query2 = "what are dogs known for?"
answer2 = rag_pipeline(query2, documents)
print(answer2)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cpu


loyal
